In [1]:
import numpy as np
import tensorflow as tf
import tf_emddistance
from scipy.spatial import distance_matrix
from scipy.optimize import linear_sum_assignment
from ipywidgets import FloatProgress
from IPython.display import display

/home/marcus/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import pickle
def load_object(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

In [3]:
point_cloud_0 = np.array(load_object("../teapot.cloud"))
point_cloud_1 = np.array(load_object("../violin_case.cloud"))
np.random.shuffle(point_cloud_0)
np.random.shuffle(point_cloud_1)
point_cloud_0 = point_cloud_0[:10]
point_cloud_1 = point_cloud_1[:10]

In [12]:
# Souurce: https://github.com/amirbawab/Hungarian-Algorithm/blob/master/Hungarian.java
def hungarian(cost_matrix):
    # Copy cost matrix
    cost_matrix = np.copy(cost_matrix)
    
    # Subtract row minima
    for row in range(cost_matrix.shape[0]):
        cost_matrix[row, :] -= np.min(cost_matrix[row, :])
    
    # Subtract column minima
    for col in range(cost_matrix.shape[1]):
        cost_matrix[:, col] -= np.min(cost_matrix[:, col])
    
    # Find which direction (vertical or horizontal) has the most zeroes
    # Negative means horizontal, positive means vertical
    def max_vh(row, col):
        r = 0
        for i in range(cost_matrix.shape[0]):
            if cost_matrix[i, col] == 0: r += 1
            if cost_matrix[row, i] == 0: r -= 1
        return r
    
    def color_neighbours(row, col, vh):
        global num_lines, lines
        if lines[row, col] == 2: return # Cell was colored twice (intersection)
        if vh >  0 and lines[row, col] ==  1: return # Don't recolor as vertical if colored vertically
        if vh <= 0 and lines[row, col] == -1: return # Don't recolor as horizontal if colored horizontally
        
        for i in range(cost_matrix.shape[0]):
            if vh > 0:
                lines[i, col] = 2 if lines[i, col] == 2 else (2 if lines[i, col] == -1 else  1)
            else:
                lines[row, i] = 2 if lines[row, i] == 2 else (2 if lines[row, i] ==  1 else -1)
        num_lines += 1
    
    # Cover all zeroes with minimum number of lines
    def cover_zeros():
        global num_lines, lines
        num_lines = 0
        lines = np.zeros(cost_matrix.shape)
        for row in range(cost_matrix.shape[0]):
            for col in range(cost_matrix.shape[1]):
                if cost_matrix[row, col] == 0:
                    color_neighbours(row, col, max_vh(row, col))
    
    progress = FloatProgress(min=0, max=cost_matrix.shape[0]); display(progress)
    while True:
        # Cover zeros
        cover_zeros()
        
        # Break if all rows and columns are covered
        progress.value = num_lines
        progress.description = "%i/%i" % (num_lines, cost_matrix.shape[0])
        print("Lines %i" %num_lines)
        print(lines)
        print()
        
        #print("Cost")
        #print(cost_matrix)
        #print()
        if num_lines >= cost_matrix.shape[0]:
            break
        
        # Subtract the minimum uncovered value
        # from all the uncovered values
        non_covered_ind = np.where(lines == 0)
        min_uncovered = np.min(cost_matrix[non_covered_ind])
        cost_matrix[non_covered_ind] -= min_uncovered
        
        # Add the minimum uncovered value
        # to the intersection values
        intersection_ind = np.where(lines == 2)
        cost_matrix[intersection_ind] += min_uncovered
    
    row_assignment = [None] * cost_matrix.shape[0]
    assigned_cols = [False] * cost_matrix.shape[1]
    
    def assign_row(row):
        if row == len(row_assignment): return True # All rows were assigned to a column
        for col in range(cost_matrix.shape[1]):
            if assigned_cols[col] == False and cost_matrix[row, col] == 0:
                row_assignment[row] = col
                assigned_cols[col] = True
                if assign_row(row+1):
                    return True
                assigned_cols[col] = False
        #print("No assignment found for row %i" %row)
        return False
        
    assign_row(0)
    
    return list(range(cost_matrix.shape[0])), row_assignment
    
#cost = np.array([[82, 83, 69, 92],
#                 [77, 37, 49, 92],
#                 [11, 69, 5, 86],
#                 [8, 9, 98, 23]])

"""cost=np.array([[68, 38, 9, 60, 46, 58, 83, 87, 84, 20], 
[53, 4, 16, 4, 44, 72, 34, 97, 69, 5], 
[21, 2, 45, 55, 34, 15, 2, 13, 12, 51], 
[8, 93, 22, 66, 25, 9, 59, 71, 12, 95], 
[42, 35, 33, 23, 3, 8, 8, 50, 23, 95], 
[74, 37, 15, 21, 36, 49, 80, 55, 79, 53], 
[21, 97, 55, 12, 25, 67, 10, 65, 2, 49], 
[8, 48, 1, 92, 8, 76, 41, 32, 87, 36], 
[32, 73, 71, 47, 94, 92, 16, 97, 5, 4], 
[58, 37, 54, 52, 84, 16, 34, 5, 72, 26]])"""

"""row_ind, col_ind = linear_sum_assignment(cost)
print(row_ind, col_ind)
print(cost[row_ind, col_ind].sum())

row_ind, col_ind = hungarian(cost)
print(row_ind, col_ind)
print(cost[row_ind, col_ind].sum())"""

'row_ind, col_ind = linear_sum_assignment(cost)\nprint(row_ind, col_ind)\nprint(cost[row_ind, col_ind].sum())\n\nrow_ind, col_ind = hungarian(cost)\nprint(row_ind, col_ind)\nprint(cost[row_ind, col_ind].sum())'

In [13]:
D = distance_matrix(point_cloud_0, point_cloud_1)

In [14]:
row_ind, col_ind = linear_sum_assignment(D)
print(row_ind, col_ind)
print(D[row_ind, col_ind].sum())

[0 1 2 3 4 5 6 7 8 9] [3 0 2 4 5 7 1 8 6 9]
529.4415292656819


In [15]:
row_ind, col_ind = hungarian(D)
print(row_ind, col_ind)
print(D[row_ind, col_ind].sum())

A Jupyter Widget

Lines 5
[[ 0.  0.  0.  0.  1.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  1.  0.]
 [-1. -1. -1. -1.  2. -1. -1. -1.  2. -1.]
 [-1. -1. -1. -1.  2. -1. -1. -1.  2. -1.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  1.  0.]
 [-1. -1. -1. -1.  2. -1. -1. -1.  2. -1.]]

Lines 6
[[ 0.  0.  0.  0.  1.  0.  0.  0.  1.  0.]
 [-1. -1. -1. -1.  2. -1. -1. -1.  2. -1.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  1.  0.]
 [-1. -1. -1. -1.  2. -1. -1. -1.  2. -1.]
 [-1. -1. -1. -1.  2. -1. -1. -1.  2. -1.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  1.  0.]
 [-1. -1. -1. -1.  2. -1. -1. -1.  2. -1.]]

Lines 7
[[ 1.  0.  0.  0.  1.  0.  0.  0.  1.  0.]
 [ 1.  0.  0.  0.  1.  0.  0.  0.  1.  0.]
 [ 1.  0.  0.  0.  1.  0. 

Lines 8
[[ 0.  0.  0.  0.  1.  0.  0.  0.  0.  1.]
 [-1. -1. -1. -1.  2. -1. -1. -1. -1.  2.]
 [-1. -1. -1. -1.  2. -1. -1. -1. -1.  2.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  1.]
 [-1. -1. -1. -1.  2. -1. -1. -1. -1.  2.]
 [-1. -1. -1. -1.  2. -1. -1. -1. -1.  2.]
 [-1. -1. -1. -1.  2. -1. -1. -1. -1.  2.]
 [-1. -1. -1. -1.  2. -1. -1. -1. -1.  2.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  1.]]

Lines 9
[[ 0.  0.  1.  0.  1.  0.  1.  0.  0.  0.]
 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]
 [ 0.  0.  1.  0.  1.  0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  1.  0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  1.  0.  1.  0.  0.  0.]
 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]
 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]
 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]
 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]
 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]]

Lines 8
[[ 0.  0.  0.  0.  1.  0.  0.  0.  0.  1.]
 [-1. -1. -1. -1.  2. -1. -1. -1. -1.  2.]
 [-1. -1. -1. -1.  2. -1. 

 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  1.]]

Lines 9
[[ 0.  0.  1.  0.  1.  0.  1.  0.  0.  0.]
 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]
 [ 0.  0.  1.  0.  1.  0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  1.  0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  1.  0.  1.  0.  0.  0.]
 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]
 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]
 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]
 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]
 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]]

Lines 8
[[ 0.  0.  0.  0.  1.  0.  0.  0.  0.  1.]
 [-1. -1. -1. -1.  2. -1. -1. -1. -1.  2.]
 [-1. -1. -1. -1.  2. -1. -1. -1. -1.  2.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  1.]
 [-1. -1. -1. -1.  2. -1. -1. -1. -1.  2.]
 [-1. -1. -1. -1.  2. -1. -1. -1. -1.  2.]
 [-1. -1. -1. -1.  2. -1. -1. -1. -1.  2.]
 [-1. -1. -1. -1.  2. -1. -1. -1. -1.  2.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  1.]]

Lines 9
[[ 0.  0.  1.  0.  1.  0.  1.  0.  0.  0.]
 [-1. -1.  2. -1.  2. -1

Lines 9
[[ 0.  0.  1.  0.  1.  0.  1.  0.  0.  0.]
 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]
 [ 0.  0.  1.  0.  1.  0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  1.  0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  1.  0.  1.  0.  0.  0.]
 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]
 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]
 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]
 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]
 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]]

Lines 8
[[ 0.  0.  0.  0.  1.  0.  0.  0.  0.  1.]
 [-1. -1. -1. -1.  2. -1. -1. -1. -1.  2.]
 [-1. -1. -1. -1.  2. -1. -1. -1. -1.  2.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  1.]
 [-1. -1. -1. -1.  2. -1. -1. -1. -1.  2.]
 [-1. -1. -1. -1.  2. -1. -1. -1. -1.  2.]
 [-1. -1. -1. -1.  2. -1. -1. -1. -1.  2.]
 [-1. -1. -1. -1.  2. -1. -1. -1. -1.  2.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  1.]]

Lines 9
[[ 0.  0.  1.  0.  1.  0.  1.  0.  0.  0.]
 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]
 [ 0.  0.  1.  0.  1.  0. 

...

 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]]

Lines 8
[[ 0.  0.  0.  0.  1.  1.  0.  0.  0.  0.]
 [-1. -1. -1. -1.  2.  2. -1. -1. -1. -1.]
 [-1. -1. -1. -1.  2.  2. -1. -1. -1. -1.]
 [ 0.  0.  0.  0.  1.  1.  0.  0.  0.  0.]
 [-1. -1. -1. -1.  2.  2. -1. -1. -1. -1.]
 [-1. -1. -1. -1.  2.  2. -1. -1. -1. -1.]
 [ 0.  0.  0.  0.  1.  1.  0.  0.  0.  0.]
 [-1. -1. -1. -1.  2.  2. -1. -1. -1. -1.]
 [ 0.  0.  0.  0.  1.  1.  0.  0.  0.  0.]
 [-1. -1. -1. -1.  2.  2. -1. -1. -1. -1.]]

Lines 9
[[ 0.  0.  1.  0.  1.  0.  1.  0.  0.  0.]
 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]
 [ 0.  0.  1.  0.  1.  0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  1.  0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  1.  0.  1.  0.  0.  0.]
 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]
 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]
 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]
 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]
 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]]

Lines 8
[[ 0.  0.  0.  0.  1.  1.  0.  0.  0.  0.]
 [-1. -1. -1. -1.  2.  2

[[ 0.  0.  1.  0.  1.  0.  1.  0.  0.  0.]
 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]
 [ 0.  0.  1.  0.  1.  0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  1.  0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  1.  0.  1.  0.  0.  0.]
 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]
 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]
 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]
 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]
 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]]

Lines 8
[[ 0.  0.  0.  0.  1.  1.  0.  0.  0.  1.]
 [-1. -1. -1. -1.  2.  2. -1. -1. -1.  2.]
 [-1. -1. -1. -1.  2.  2. -1. -1. -1.  2.]
 [ 0.  0.  0.  0.  1.  1.  0.  0.  0.  1.]
 [-1. -1. -1. -1.  2.  2. -1. -1. -1.  2.]
 [-1. -1. -1. -1.  2.  2. -1. -1. -1.  2.]
 [ 0.  0.  0.  0.  1.  1.  0.  0.  0.  1.]
 [-1. -1. -1. -1.  2.  2. -1. -1. -1.  2.]
 [ 0.  0.  0.  0.  1.  1.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  1.  1.  0.  0.  0.  1.]]

Lines 9
[[ 0.  0.  1.  0.  1.  0.  1.  0.  0.  0.]
 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]
 [ 0.  0.  1.  0.  1.  0.  1.  0. 

Lines 9
[[ 0.  0.  1.  0.  1.  0.  1.  0.  0.  0.]
 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]
 [ 0.  0.  1.  0.  1.  0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  1.  0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  1.  0.  1.  0.  0.  0.]
 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]
 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]
 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]
 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]
 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]]

Lines 8
[[ 0.  0.  0.  0.  1.  1.  0.  0.  0.  1.]
 [-1. -1. -1. -1.  2.  2. -1. -1. -1.  2.]
 [-1. -1. -1. -1.  2.  2. -1. -1. -1.  2.]
 [ 0.  0.  0.  0.  1.  1.  0.  0.  0.  1.]
 [-1. -1. -1. -1.  2.  2. -1. -1. -1.  2.]
 [-1. -1. -1. -1.  2.  2. -1. -1. -1.  2.]
 [ 0.  0.  0.  0.  1.  1.  0.  0.  0.  1.]
 [-1. -1. -1. -1.  2.  2. -1. -1. -1.  2.]
 [ 0.  0.  0.  0.  1.  1.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  1.  1.  0.  0.  0.  1.]]

Lines 9
[[ 0.  0.  1.  0.  1.  0.  1.  0.  0.  0.]
 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]
 [ 0.  0.  1.  0.  1.  0. 

 [ 0.  0.  0.  0.  1.  1.  0.  0.  0.  1.]]

Lines 9
[[ 0.  0.  1.  0.  1.  0.  1.  0.  0.  0.]
 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]
 [ 0.  0.  1.  0.  1.  0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  1.  0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  1.  0.  1.  0.  0.  0.]
 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]
 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]
 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]
 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]
 [-1. -1.  2. -1.  2. -1.  2. -1. -1. -1.]]

Lines 8
[[ 0.  0.  0.  0.  1.  1.  0.  0.  0.  1.]
 [-1. -1. -1. -1.  2.  2. -1. -1. -1.  2.]
 [-1. -1. -1. -1.  2.  2. -1. -1. -1.  2.]
 [ 0.  0.  0.  0.  1.  1.  0.  0.  0.  1.]
 [-1. -1. -1. -1.  2.  2. -1. -1. -1.  2.]
 [-1. -1. -1. -1.  2.  2. -1. -1. -1.  2.]
 [ 0.  0.  0.  0.  1.  1.  0.  0.  0.  1.]
 [-1. -1. -1. -1.  2.  2. -1. -1. -1.  2.]
 [ 0.  0.  0.  0.  1.  1.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  1.  1.  0.  0.  0.  1.]]

Lines 9
[[ 0.  0.  1.  0.  1.  0.  1.  0.  0.  0.]
 [-1. -1.  2. -1.  2. -1

KeyboardInterrupt: 